In [1]:
import numpy as np 
import pandas as pd 
import random
from keras.layers import merge
from keras.layers.recurrent import LSTM
import keras.backend as K
from keras.layers import BatchNormalization,concatenate,Flatten,Concatenate,Activation,Conv1D,Add ,MaxPooling1D,Masking,GRU
from keras.layers.core import Lambda, Dense, Activation
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam,RMSprop    
from keras.models import Model
from keras.layers import Input,Dropout,GlobalMaxPooling1D
from keras.callbacks import Callback,EarlyStopping

import tensorflow as tf
from tensorflow import set_random_seed   

/data/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [52]:
batch_size=64
alphaHiddenDimSize=12
betaHiddenDimSize=12
embDimSize=6
timeseq=32
is_attention=True

x =np.random.random((3200,32,6)).astype(np.float32)
y =np.vstack([np.zeros((1600,1)),np.ones((1600,1))]).astype(np.float32)


w_alpha = tf.Variable(tf.random_normal([alphaHiddenDimSize, 1]),name='w_alpha')
b_alpha = tf.Variable(tf.zeros([1]),name='b_alpha')
w_beta = tf.Variable(tf.random_normal([betaHiddenDimSize, embDimSize]),name='w_beta') 
b_beta =tf.Variable(tf.zeros([embDimSize]),name='b_beta')


data   = tf.placeholder(tf.float32, [None,timeseq, embDimSize],name='data')
label  = tf.placeholder(tf.float32, [None,1],name='target') 

if is_attention:
    data_reverse = data[:,::-1,:]
    reverse_h_a = GRU(alphaHiddenDimSize, return_sequences=True)(data_reverse)
    reverse_h_a = reverse_h_a[:,::-1,:]*0.5
    reverse_h_b = GRU(betaHiddenDimSize, return_sequences=True)(data_reverse)
    reverse_h_b = reverse_h_b[:,::-1,:]*0.5
    alpha_total=[]
    beta_total=[]
    for i in range(timeseq):
        reverse_a_time=reverse_h_a[:,i,:]   
        preAlpha = tf.matmul(reverse_a_time,w_alpha)+b_alpha
        alpha_total.append(preAlpha)

        reverse_b_time=reverse_h_b[:,i,:]
        prebeta = tf.matmul(reverse_b_time,w_beta)+b_beta
        prebeta = Activation('tanh')(prebeta) 
        beta_total.append(prebeta)
        
    outlist = tf.reshape(tf.stack(alpha_total, axis=1),(-1,timeseq))
    alpha    = Activation('softmax')(outlist)  
    beta = tf.stack(beta_total, axis=1)
    c_t = (alpha[:,:,None]* beta * data)
    c_t = K.sum(c_t,axis=1)
    y_hat = Dense(1)(c_t)
else:
    c_t = GRU(alphaHiddenDimSize, return_sequences=False)(data)
    y_hat = Dense(1)(c_t)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=label,logits=y_hat))
solver = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(loss) 


sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

total_batch=int(x.shape[0]/batch_size)
total_cost=0

attention_total=np.zeros((0,32,6))
for i in range(total_batch):
    x_batch=x[i*batch_size:(i+1)*batch_size]
    y_batch=y[i*batch_size:(i+1)*batch_size]
    
    if is_attention:
        _,current_loss,current_alpha,current_beta=sess.run([solver, loss,alpha,beta],  
                                           feed_dict={data:x_batch,label:y_batch}) 

        attention_weight=current_alpha[:,:,None]* current_beta
        attention_total=np.vstack([attention_total,attention_weight])
    else:   
        _,current_loss=sess.run([solver, loss], feed_dict={data:x_batch,label:y_batch})     
    
    total_cost+=current_loss
    avg_cost=total_cost/(i+1)

#查看平均注意力
# avg_attention_weight=np.mean(attention_total,axis=0)
print(avg_cost)

/data/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1719: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0.7045434153079987
